# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,semirom,31.4167,51.5667,17.37,54,40,1.78,IR,Tue Aug 8 21:56:16 2023
1,1,yellowknife,62.4560,-114.3525,20.29,45,75,5.66,CA,Tue Aug 8 21:56:16 2023
2,2,waitangi,-43.9535,-176.5597,9.34,82,100,1.34,NZ,Tue Aug 8 21:56:16 2023
3,3,ushuaia,-54.8000,-68.3000,1.81,75,0,1.54,AR,Tue Aug 8 21:51:25 2023
4,4,iqaluit,63.7506,-68.5145,4.85,86,20,4.12,CA,Tue Aug 8 21:53:05 2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_1_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 500,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City"
    )

# Display the map
map_1_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df["Max Temp"] = city_data_df["Max Temp"].astype('int')

ideal_weather = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_clean = ideal_weather.dropna()

# Display sample data
ideal_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,83,paphos,34.7667,32.4167,23,78,0,3.09,CY,Tue Aug 8 21:56:46 2023
110,110,alice springs,-23.7000,133.8833,23,22,0,3.60,AU,Tue Aug 8 21:54:33 2023
130,130,shahrud,36.4182,54.9763,23,41,0,1.96,IR,Tue Aug 8 21:57:03 2023
136,136,ghadamis,30.1337,9.5007,26,37,0,4.10,LY,Tue Aug 8 21:57:05 2023
177,177,emerald,-23.5333,148.1667,22,37,0,3.92,AU,Tue Aug 8 21:54:06 2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_clean.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
83,83,paphos,34.7667,32.4167,23,78,0,3.09,CY,Tue Aug 8 21:56:46 2023,
110,110,alice springs,-23.7000,133.8833,23,22,0,3.60,AU,Tue Aug 8 21:54:33 2023,
130,130,shahrud,36.4182,54.9763,23,41,0,1.96,IR,Tue Aug 8 21:57:03 2023,
136,136,ghadamis,30.1337,9.5007,26,37,0,4.10,LY,Tue Aug 8 21:57:05 2023,
177,177,emerald,-23.5333,148.1667,22,37,0,3.92,AU,Tue Aug 8 21:54:06 2023,
202,202,al jawf,29.5000,38.7500,24,36,0,2.95,SA,Tue Aug 8 21:57:27 2023,
234,234,fuman,37.2273,49.3142,23,54,0,0.45,IR,Tue Aug 8 21:57:36 2023,
238,238,rosetta,31.4044,30.4164,24,75,0,2.78,EG,Tue Aug 8 21:54:18 2023,
260,260,al bawiti,28.3492,28.8659,23,69,0,3.22,EG,Tue Aug 8 21:57:43 2023,
364,364,zarzis,33.5040,11.1122,24,61,0,2.55,TN,Tue Aug 8 21:58:17 2023,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{latitude},{longitude},{radius}"
    params["bias"] = f"proximity:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
paphos - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
shahrud - nearest hotel: Дружба
ghadamis - nearest hotel: No hotel found
emerald - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
fuman - nearest hotel: No hotel found
rosetta - nearest hotel: Shewail Home
al bawiti - nearest hotel: No hotel found
zarzis - nearest hotel: No hotel found
belyy yar - nearest hotel: No hotel found
ijevan - nearest hotel: Маяк
ilovlya - nearest hotel: No hotel found
puerto lempira - nearest hotel: No hotel found
eufaula - nearest hotel: No hotel found
griffith - nearest hotel: No hotel found
murzuq - nearest hotel: No hotel found
akdepe - nearest hotel: No hotel found
sakakah - nearest hotel: Köprülü Mehmet Paşa Kervansarayı
douglas - nearest hotel: No hotel found
turaif - nearest hotel: No hotel found
barreiras - nearest hotel: No hotel found
zaragoza - nearest hotel: No hotel found
pocatello - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
83,83,paphos,34.7667,32.4167,23,78,0,3.09,CY,Tue Aug 8 21:56:46 2023,No hotel found
110,110,alice springs,-23.7000,133.8833,23,22,0,3.60,AU,Tue Aug 8 21:54:33 2023,No hotel found
130,130,shahrud,36.4182,54.9763,23,41,0,1.96,IR,Tue Aug 8 21:57:03 2023,Дружба
136,136,ghadamis,30.1337,9.5007,26,37,0,4.10,LY,Tue Aug 8 21:57:05 2023,No hotel found
177,177,emerald,-23.5333,148.1667,22,37,0,3.92,AU,Tue Aug 8 21:54:06 2023,No hotel found
202,202,al jawf,29.5000,38.7500,24,36,0,2.95,SA,Tue Aug 8 21:57:27 2023,No hotel found
234,234,fuman,37.2273,49.3142,23,54,0,0.45,IR,Tue Aug 8 21:57:36 2023,No hotel found
238,238,rosetta,31.4044,30.4164,24,75,0,2.78,EG,Tue Aug 8 21:54:18 2023,Shewail Home
260,260,al bawiti,28.3492,28.8659,23,69,0,3.22,EG,Tue Aug 8 21:57:43 2023,No hotel found
364,364,zarzis,33.5040,11.1122,24,61,0,2.55,TN,Tue Aug 8 21:58:17 2023,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_2_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 500,
    frame_height = 500,
    scale = 0.5,
    color = "City",
    hover_cols = ['Hotel Name', 'Country']
    
    )

# Display the map
map_2_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)